In [27]:
#imports
#@title Import & seed
import time
import re
import os
import math
import numpy as np
from matplotlib import pyplot as plt
import requests
import collections
import pickle
import copy, random
import nltk as nl
nl.download('punkt')
from itertools import zip_longest

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Input, Reshape, BatchNormalization, Dense, Dropout, concatenate,
    Embedding, LSTM, Dense, GRU, Bidirectional, Add
)
from tensorflow.keras.activations import elu, relu, softmax, sigmoid
from tensorflow.keras import regularizers
from tensorflow.keras.utils import to_categorical


#Variables
FILENAME = "divineComedyProf.txt"
CLEANFILENAME = "cleanComedyProf.txt"
INFERNO = "inferno_syllnew.txt"
PURGATORIO = "purgatorio_syllnew.txt"
PARADISO = "paradiso_syllnew.txt"
PARADISOCLEAN = "paradiso_clean.txt"


vocab_size = 1800
terces_per_batch = 4
terces_len = 75
batch_len = terces_per_batch * (terces_len + 1)


#Setup
print(tf.__version__)

np.random.seed(1234)
!nvidia-smi

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2.5.0
NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



#Clean files


In [ ]:
#count chars
char_list = []
with open(FILENAME) as file:
  while True:
    char = file.read(1)
    if not char:
      print("End of file")
      break
    char = char.lower()

    #add good char to char list
    if char not in char_list:
      char_list.append(char)

print(char_list)

End of file
['\n', 'p', 'a', 'r', 'd', 'i', 's', 'o', ' ', '•', 'c', 'n', 't', 'x', '1', '«', '|', 'v', 'e', 'g', 'm', ',', 'f', 'l', 'u', '2', 'ù', 'h', '3', '’', '4', '5', 'b', 'ì', '6', 'ò', '.', '7', '8', '9', 'è', 'q', '0', 'ï', 'z', 'à', 'é', ':', '!', '»', ';', 'ü', '“', '”']


In [ ]:
removable_chars = ['•', '—', '-', '(', ')', ',', '.', ':', ';', '“', '”', '«', '»', '"','0','1','2','3','4','5','6','7','8','9']
#‘ e ’ usate per racchiudere parti parlate in latino
#“ e ” racchiude il parlato sia in italiano che latino
#idem « e »


clean_char_list = []
removedCharsCounter = 0
fileOut = open(CLEANFILENAME, "a")

with open(FILENAME) as file:
  while True:
    char = file.read(1)
    if not char:
      print("End of file")
      break

    #transform upper case to lower case
    char = char.lower()

    #remove unwanted chars
    if char not in removable_chars:

      #add acceptable chars to char list
      if char not in clean_char_list:
        clean_char_list.append(char)
      
      #output in new file
      fileOut.write(char)
    else:
      removedCharsCounter += 1


print(char_list)
print(removedCharsCounter)

End of file
['\n', 'p', 'a', 'r', 'd', 'i', 's', 'o', ' ', '•', 'c', 'n', 't', 'x', '1', '«', '|', 'v', 'e', 'g', 'm', ',', 'f', 'l', 'u', '2', 'ù', 'h', '3', '’', '4', '5', 'b', 'ì', '6', 'ò', '.', '7', '8', '9', 'è', 'q', '0', 'ï', 'z', 'à', 'é', ':', '!', '»', ';', 'ü', '“', '”']
462


In [ ]:
f = open(CLEANFILENAME, "r")
lines = f.readlines()

number_of_syllables = []
line_counter = 1

for line in lines:
  #excludes all empty lines and canto intros
  if '|' in line:
    syllables = line.count('|')
    number_of_syllables.append(syllables)
    if syllables != 11:
      print("line number {} with text {} has {} syllables".format(line_counter,line,syllables))
    line_counter += 1

Counter(number_of_syllables)

# Tokenization

"""Once tokenized the dataset, a map of the syllables with the integer index is created. The final dimension
of the vocabulary is 1874 tokens but it is limited to 1800 to remove the tail of infrequent syllables.

We substituted every space between words with the special token < SEP > and inserted at the beginning
of each verse the token < GO >. To make all verses the same lengths, we used the special character
“< PAD >” to pad every terces to the length of 75 tokens. At the end of each verse we appended the
symbol “< EOV > “, while at the end of each sentence “< EOS >”."""

number_of_syllables = []
line_counter = 1
"""
space -> <SPA>
verst start -> <VST>
padding (up to 75) -> <PAD>
end of verse -> <EOV>
end of sentence -> <EOS>
"""


In [28]:
a = "ciccia"
a=a[2:]
print(a)

ccia


In [31]:
def get_hyp_lm_tercets(tercets):
    new_tercets = []
    for tercet in tercets:
        new_tercets.append([])
        for verse in tercet:
            new_tercets[-1].append([])
            for hyp_w in verse:
                new_tercets[-1][-1].extend(hyp_w)
                new_tercets[-1][-1].append('<SEP>')
            new_tercets[-1][-1] = new_tercets[-1][-1][:-1]

    return new_tercets

def hyphenation(verse):
    """
    Split word in syllables
    :param verse: input string
    :return: a list containing syllables of the word
    """
    syllables = []
    syllable = ""

    
    verse = verse[1:]
    verse = verse+"|" 

    for letter in verse:
        if letter != '|':
            syllable += letter
        else:
            syllables.append(syllable)
            syllable = ""
               
    return syllables



def get_dc_hyphenation(canti):
    hyp_canti, hyp_tokens = [], []
    for canto in canti:
        hyp_canti.append([])
        for verso in canto:
            syllables = hyphenation(verso)
            hyp_canti[-1].append(syllables)
            for syllable in syllables:
                hyp_tokens.extend(syllable)

    return hyp_canti, hyp_tokens
    

def get_dc_cantos(filename, encoding=None):
    # raw_data = read_words(filename=filename)
    cantos = []
    cantoCounter =  0
    cantos.append([])
    with open(filename, "r", encoding=encoding) as f:
        for line in f:
            sentence = line.strip()
            if 'canto' not in sentence and len(sentence) > 2:
                if len(sentence) > 2:
                    cantos[cantoCounter].append(sentence)
                    """
                    if len(cantos[cantoCounter]) == 3:
                      cantoCounter += 1
                      cantos.append([])
                    """
    return cantos


def create_tercets(cantos):
    tercets = []
    for i,canto in enumerate(cantos):
        for v,verse in enumerate(canto):
            if v%3 == 0:
                tercets.append([])

            tercets[-1].append(verse)
        tercets = tercets[:-1]  # removes the last malformed tercets (only 2 verses)

    return tercets

def pad_list(l, pad_token, max_l_size, keep_lasts=False, pad_right=True):
    """
    Adds a padding token to a list
    inputs:
    :param l: input list to pad.
    :param pad_token: value to add as padding.
    :param max_l_size: length of the new padded list to return,
    it truncates lists longer that 'max_l_size' without adding
    padding values.
    :param keep_lasts: If True, preserves the max_l_size last elements
    of a sequence (by keeping the same order).  E.g.:
    if keep_lasts is True and max_l_size=3 [1,2,3,4] becomes [2,3,4].


    :return: the list padded or truncated.
    """
    to_pad = []
    max_l = min(max_l_size, len(l))  # maximum len
    l_init = len(l) - max_l if len(l) > max_l and keep_lasts else 0  # initial position where to sample from the list
    l_end = len(l) if len(l) > max_l and keep_lasts else max_l
    for i in range(l_init, l_end):
        to_pad.append(l[i])

    # for j in range(len(l), max_l_size):
    #     to_pad.append(pad_token)
    pad_tokens = [pad_token] * (max_l_size-len(l))
    padded_l = to_pad + pad_tokens if pad_right else pad_tokens + to_pad

    return padded_l


def save_data(data, file):
    with open(file, 'wb') as output:
        pickle.dump(data, output, pickle.HIGHEST_PROTOCOL)

def load_data(file):
    with open(file, 'rb') as obj:
        return pickle.load(obj)

def print_and_write(file, s):
    print(s)
    file.write(s)


class Vocabulary(object):
    def __init__(self, vocab_size=None):
        self.dictionary = dict()
        self.rev_dictionary = dict()
        self.count = []
        self.special_tokens = []
        self.vocab_size = vocab_size

    def build_vocabulary_from_counts(self, count, special_tokens=[]):
        """
        Sets all the attributes of the Vocabulary object.
        :param count: a list of lists as follows: [['token', number_of_occurrences],...]
        :param special_tokens: a list of strings. E.g. ['<EOS>', '<PAD>',...]
        :return: None
        """

        dictionary = dict()
        for word, _ in count:
            dictionary[word] = len(dictionary)

        # adding eventual special tokens to the dictionary (e.g. <EOS>,<PAD> etc..)
        d = len(dictionary)
        for i, token in enumerate(special_tokens):
            dictionary[token] = d + i

        self.count = count
        self.dictionary = dictionary
        self.rev_dictionary = dict(zip(self.dictionary.values(), self.dictionary.keys()))
        self.special_tokens = special_tokens
        self.vocab_size = len(dictionary)

    def build_vocabulary_from_tokens(self, tokens, vocabulary_size=None, special_tokens=[]):
        """
        Given a list of tokens, it sets the Vocabulary object attributes by constructing
        a dictionary mapping each token to a unique id.
        :param tokens: a list of strings.
         E.g. ["the", "cat", "is", ... ".", "the", "house" ,"is" ...].
         NB: Here you should put all your token instances of the corpus.
        :param vocabulary_size: The number of elements of your vocabulary. If there are more
        than 'vocabulary_size' elements on tokens, it considers only the 'vocabulary_size'
        most frequent ones.
        :param special_tokens: Optional. A list of strings. Useful to add special tokens in vocabulary.
        If you don't have any, keep it empty.
        :return: None
        """

        vocabulary_size = vocabulary_size if vocabulary_size is not None else self.vocab_size
        vocabulary_size = vocabulary_size - (len(special_tokens) + 1) if vocabulary_size else None
        # counts occurrences of each token
        count = [['<UNK>', -1]]
        count.extend(collections.Counter(tokens).most_common(vocabulary_size))  # takes only the most frequent ones, if size is None takes them all
        self.build_vocabulary_from_counts(count, special_tokens)  # actually build the vocabulary
        self._set_unk_count(tokens)  # set the number of OOV instances

    @staticmethod
    def merge_vocabulary(vocab0, vocab1, vocabulary_size=-1):
        """
        Merge two Vocabulary objects into a new one.
        :param vocab0: first Vocabulary object
        :param vocab1: second Vocabulary object
        :param vocabulary_size: parameter to decide the merged vocabulary size.
        With default value -1, all the words of both vocabularies are preserved.
        When set to 0, the size of the vocabulary is set to the size of vocab0,
        when set to 1 it is kept the size of vocab1.
        :return: a new vocabulary
        """
        # get size of the new vocabulary
        vocab_size = vocab0.vocab_size + vocab1.vocab_size if vocabulary_size == -1 else vocabulary_size
        merged_special_tokens = list(set(vocab0.special_tokens) | set(vocab1.special_tokens))

        # merge the counts from the two vocabularies and then selects the most_common tokens
        merged_counts = collections.Counter(dict(vocab0.count)) + collections.Counter(dict(vocab1.count))
        merged_counts = merged_counts.most_common(vocab_size)
        count = [['<UNK>', -1]]
        count.extend(merged_counts)

        # create the new vocabulary
        merged_vocab = Vocabulary(vocab_size)
        merged_vocab.build_vocabulary_from_counts(count, merged_special_tokens)
        return merged_vocab

    @staticmethod
    def merge_vocabularies(vocab_list, vocab_size=None):
        """
        Join a list of vocabularies into a new one.
        :param vocab_list: a list of Vocabulary objects
        :param vocab_size: the maximum size of the merged vocabulary.
        :return: a vocabulary merging them all.
        """
        vocab_size = vocab_size if vocab_size else sum([v.vocab_size for v in vocab_list])
        merged_vocab = Vocabulary(vocab_size)
        for voc in vocab_list:
            merged_vocab = Vocabulary.merge_vocabulary(merged_vocab, voc, vocab_size)
        return merged_vocab

    def string2id(self, dataset):
        """
        Converts a dataset of strings into a dataset of ids according to the object dictionary.
        :param dataset: any string-based dataset with any nested lists.
        :return: a new dataset, with the same shape of dataset, where each string is mapped into its
        corresponding id associated in the dictionary (0 for unknown tokens).
        """

        def _recursive_call(items):
            new_items = []
            for item in items:
                if isinstance(item, str) or isinstance(item, int) or isinstance(item, float):
                    new_items.append(self.word2id(item))
                else:
                    new_items.append(_recursive_call(item))
            return new_items

        return _recursive_call(dataset)

    def id2string(self, dataset):
        """
        Converts a dataset of integer ids into a dataset of string according to the reverse dictionary.
        :param dataset: any int-based dataset with any nested lists. Allowed types are int, np.int32, np.int64.
        :return: a new dataset, with the same shape of dataset, where each token is mapped into its
        corresponding string associated in the reverse dictionary.
        """
        def _recursive_call(items):
            new_items = []
            for item in items:
                if isinstance(item, int) or isinstance(item, np.int) or isinstance(item, np.int32) or isinstance(item, np.int64):
                    new_items.append(self.id2word(item))
                else:
                    new_items.append(_recursive_call(item))
            return new_items

        return _recursive_call(dataset)

    def word2id(self, item):
        """
        Maps a string token to its corresponding id.
        :param item: a string.
        :return: If the token belongs to the vocabulary, it returns an integer id > 0, otherwise
        it returns the value associated to the unknown symbol, that is typically 0.
        """
        return self.dictionary[item] if item in self.dictionary else self.dictionary['<UNK>']

    def id2word(self, token_id):
        """
        Maps an integer token to its corresponding string.
        :param token_id: an integer.
        :return: If the id belongs to the vocabulary, it returns the string
        associated to it, otherwise it returns the string associated
        to the unknown symbol, that is '<UNK>'.
        """

        return self.rev_dictionary[token_id] if token_id in self.rev_dictionary else self.rev_dictionary[self.dictionary['<UNK>']]

    def get_unk_count(self):
        return self.count[0][1]

    def _set_unk_count(self, tokens):
        """
        Sets the number of OOV instances in the tokens provided
        :param tokens: a list of tokens
        :return: None
        """
        data = list()
        unk_count = 0
        for word in tokens:
            if word in self.dictionary:
                index = self.dictionary[word]
            else:
                index = 0  # dictionary['<UNK>']
                unk_count += 1
            data.append(index)
        self.count[0][1] = unk_count

    def add_element(self, name, is_special_token=False):
        if name not in self.dictionary:
            self.vocab_size += 1
            self.dictionary[name] = self.vocab_size
            self.rev_dictionary[self.vocab_size] = name

            if is_special_token:
                self.special_tokens = list(self.special_tokens)
                self.special_tokens.append(name)

            self.count.append([name, 1])

    def set_vocabulary(self, dictionary, rev_dictionary, special_tokens, vocab_size):
        self.dictionary = dictionary,
        self.rev_dictionary = rev_dictionary
        self.special_tokens = special_tokens
        self.vocab_size = vocab_size

    @staticmethod
    def load_vocabulary(filename):
        return load_data(filename)

    def save_vocabulary(self, filename):
        save_data(self, filename)

class SyLMDataset(object):
    def __init__(self, config, sy_vocab=None):
        self.config = config
        self.vocabulary = sy_vocab

        self.raw_train_x = []
        self.raw_val_x = []
        self.raw_test_x = []
        self.raw_x = []

        self.train_x, self.train_y = [], []
        self.val_x, self.val_y = [], []
        self.test_x, self.test_y = [], []
        self.x, self.y = [], []

    def initialize(self, sess):
        pass

    def load(self, sources):
        """
        Extract raw texts form sources and gather them all together.
        :param sources: a string or an iterable of strings containing the file(s)
        to process in order to build the dataset.
        :return: a list of raw strings.
        """
        return NotImplementedError

    def build(self, sources, split_size=0.8):
        """
        :param sources: a string or an iterable of strings containing the file(s)
        to process in order to build the dataset.
        :param split_size: the size to split the dataset, set >=1.0 to not split.
        """

        raw_x = self.load(sources)
        # raw_x = self.tokenize([self.preprocess(ex) for ex in raw_x])  # fixme
        # splitting data
        self.raw_x = raw_x
        if split_size < 1.0:
            self.raw_train_x, self.raw_test_x = self.split(self.raw_x, train_size=split_size)
            self.raw_train_x, self.raw_val_x = self.split(self.raw_train_x, train_size=split_size)
        else:
            self.raw_train_x = self.raw_x

        if self.vocabulary is None:
            # creates vocabulary
            tokens = [item for sublist in self.raw_train_x for item in sublist]  # get tokens
            special_tokens = ("<GO>", "<PAD>", "<SEP>", "<EOS>", "<EOV>")
            self._create_vocab(tokens, special_tokens=special_tokens)

        # creates x,y for train
        self.train_x = self._build_dataset(self.raw_train_x, insert_go=True, max_len=self.config.sentence_max_len, shuffle=False)
        self.train_y = self._build_dataset(self.raw_train_x, insert_go=True, max_len=self.config.sentence_max_len, shuffle=False)

        # creates x,y for validation
        self.val_x = self._build_dataset(self.raw_val_x, insert_go=True, max_len=self.config.sentence_max_len, shuffle=False)
        self.val_y = self._build_dataset(self.raw_val_x, insert_go=True, max_len=self.config.sentence_max_len, shuffle=False)

        # creates x,y for testing
        self.test_x = self._build_dataset(self.raw_test_x, insert_go=True, max_len=self.config.sentence_max_len, shuffle=False)
        self.test_y = self._build_dataset(self.raw_test_x, insert_go=True, max_len=self.config.sentence_max_len, shuffle=False)

    def _create_vocab(self, tokens, special_tokens=("<PAD>", "<GO>", "<SEP>", "<EOV>", "<EOS>")):
        """
        Create the vocabulary. Special tokens can be added to the tokens obtained from
        the corpus.
        :param tokens: a list of all the tokens in the corpus. Each token is a string.
        :param special_tokens: a list of strings.
        """
        print("creating_vocabulary") 

        vocab = Vocabulary(vocab_size=self.config.input_vocab_size)
        vocab.build_vocabulary_from_tokens(tokens, special_tokens=special_tokens)
        self.vocabulary = vocab

    @staticmethod
    def split(raw_data, train_size=0.8):
        size = math.floor(len(raw_data)*train_size)
        return raw_data[:size], raw_data[size:]

    @staticmethod
    def preprocess(txt):
        return txt

    @staticmethod
    def shuffle(x):
        return random.sample(x, len(x))

    @staticmethod
    def tokenize(txt):
        return txt

    def _build_dataset(self, raw_data, max_len=100, insert_go=True, keep_lasts=False, pad_right=True, shuffle=True):
        """
        Converts all the tokens in e1_raw_data by mapping each token with its corresponding
        value in the dictionary. In case of token not in the dictionary, they are assigned to
        a specific id. Each sequence is padded up to the seq_max_len setup in the config.

        :param raw_data: list of sequences, each sequence is a list of tokens (strings).
        :param max_len: max length of a sequence, crop longer and pad smaller ones.
        :param insert_go: True to insert <GO>, False otherwise.
        :param keep_lasts: True to truncate initial elements of a sequence.
        :param pad_right: pad to the right (default value True), otherwise pads to left.
        :param shuffle: Optional. If True data are shuffled.
        :return: A list of sequences where each token in each sequence is an int id.
        """
        dataset = []
        for sentence in raw_data:
            sentence_ids = [self.vocabulary.word2id("<GO>")] if insert_go else []
            sentence_ids.extend([self.vocabulary.word2id(w) for w in sentence])
            sentence_ids.append(self.vocabulary.word2id("<EOS>"))
            sentence_ids = pad_list(sentence_ids, self.vocabulary.word2id("<PAD>"), max_len, keep_lasts=keep_lasts, pad_right=pad_right)

            dataset.append(sentence_ids)

        if shuffle:
            return random.sample(dataset, len(dataset))
        else:
            return dataset

    def get_batches(self, batch_size=32, split_sel='train'):
        """
        Iterator over the training set. Useful method to run experiments.
        :param batch_size: size of the mini_batch
        :return: input and target.
        """
        if split_sel == 'train':
            x, y = self.train_x, self.train_y
        elif split_sel == 'val':
            x, y = self.val_x, self.val_y
        else:
            x, y = self.test_x, self.test_y
        
        i = 0#random.randint(0, batch_size)
        batches = []
        eov = self.vocabulary.word2id("<EOV>")
        go = self.vocabulary.word2id("<GO>")
        # prepare batches
        while i < len(x):
            j = 0
            batch_x, batch_y = [], []
            while j < batch_size and i+j<len(x):
                for c in x[i+j]:
                  batch_x.append(c)
                batch_x.append(eov)
                for c in y[i+j]:
                  batch_y.append(c)
                batch_y.append(eov)
                j += 1
            i += batch_size
            batches.append((batch_x, batch_y))

        # supply
        i = 0
        while i < len(batches):
            yield batches[i][0], batches[i][1]
            i += 1

class DanteSyLMDataset(SyLMDataset):
    def __init__(self, config, sy_vocab=None):
        """
        Class to create a dataset from Dante Alighieri's Divine Comedy.
        :param config: a Config object
        :param sy_vocab: (optional) a Vocabulary object where tokens of the dictionary
        are syllables. If None, the vocabulary is create automatically from the source.
        """
        super().__init__(config, sy_vocab)

    def load(self, sources):
        """
        Load examples from dataset
        :param sources: data filepath.
        :return:
        """
        canti = get_dc_cantos(filename=sources)  # get raw data from file
        canti, tokens = get_dc_hyphenation(canti)  # converts each

        tercets = create_tercets(canti)
        tercets = get_hyp_lm_tercets(tercets)

        x = []
        for tercet in tercets:
            x.append([])
            for verse in tercet:
                x[-1].extend(verse)
                x[-1].append("<EOV>")

        #x = self.shuffle(x)
        return x

def seq2str(seq):
    def output2string(batch, rev_vocabulary, special_tokens, end_of_tokens):
        to_print = ''
        for token in batch:
            if token in special_tokens:
                to_print += ' '
            elif end_of_tokens and token in end_of_tokens:
                to_print += '\n'
            elif token in rev_vocabulary:
                to_print += rev_vocabulary[token]
            else:
                to_print += '<UNK>'
        return to_print

    return output2string(seq, poetry_sy_lm_dataset.vocabulary.rev_dictionary,
      special_tokens=[poetry_sy_lm_dataset.vocabulary.word2id("<PAD>"), 0, poetry_sy_lm_dataset.vocabulary.word2id("<SEP>"),
                      poetry_sy_lm_dataset.vocabulary.word2id("<GO>"), poetry_sy_lm_dataset.vocabulary.word2id("<EOS>")],
      end_of_tokens=[poetry_sy_lm_dataset.vocabulary.word2id("<EOV>")])

class cnfg:
  vocab_size = vocab_size
  input_vocab_size = vocab_size
  sentence_max_len = terces_len

config = cnfg()
poetry_sy_lm_dataset = DanteSyLMDataset(config, sy_vocab=None)

data_path = CLEANFILENAME  # dataset location, here just the name of the source file

poetry_sy_lm_dataset.build(data_path, split_size=0.99)  # actual creation of  vocabulary (if not provided) and dataset
print("Train size: " + str(len(poetry_sy_lm_dataset.train_y)))
print("Val size: " + str(len(poetry_sy_lm_dataset.val_y)))
print("Test size: " + str(len(poetry_sy_lm_dataset.test_y)))

eov = poetry_sy_lm_dataset.vocabulary.word2id("<EOV>")
pad = poetry_sy_lm_dataset.vocabulary.word2id("<PAD>")
go = poetry_sy_lm_dataset.vocabulary.word2id("<GO>")
eos = poetry_sy_lm_dataset.vocabulary.word2id("<EOS>")

batches = [b for b in poetry_sy_lm_dataset.get_batches(terces_per_batch)]
print(batches[0][0])
print(batches[0][1])
print(len(batches[0][0]))
test_b = [b for b in poetry_sy_lm_dataset.get_batches(terces_per_batch, split_sel='test')]
print(test_b[0][0])
print(test_b[0][1])
print(len(test_b[0][0]))
val_b = [b for b in poetry_sy_lm_dataset.get_batches(terces_per_batch, split_sel='val')]
print(val_b[0][0])
print(val_b[0][1])
print(len(val_b[0][0]))
len(poetry_sy_lm_dataset.vocabulary.dictionary.items())

creating_vocabulary
Train size: 4649
Val size: 47
Test size: 48
dict_items([('<UNK>', 0), ('<SEP>', 47), (' ', 2), ('e', 3), ('a', 4), ('i', 5), ('o', 6), ('n', 7), ('r', 8), ('l', 9), ('t', 10), ('s', 11), ('c', 12), ('d', 13), ('<EOV>', 49), ('u', 15), ('m', 16), ('p', 17), ('v', 18), ('’', 19), ('g', 20), ('h', 21), ('f', 22), ('q', 23), ('b', 24), ('z', 25), (';', 26), ('ì', 27), ('ù', 28), ('ò', 29), ('è', 30), ('é', 31), ('à', 32), ('ï', 33), ('?', 34), ('!', 35), ('ë', 36), ('ü', 37), ('ó', 38), ('ä', 39), ('‘', 40), ('x', 41), ('j', 42), ('ö', 43), ('y', 44), ('<GO>', 45), ('<PAD>', 46), ('<EOS>', 48)])
[45, 7, 3, 9, 2, 47, 16, 3, 25, 47, 25, 6, 2, 47, 13, 3, 9, 2, 47, 12, 4, 16, 47, 16, 5, 7, 2, 47, 13, 5, 2, 47, 7, 6, 47, 11, 10, 8, 4, 2, 47, 18, 5, 47, 10, 4, 49, 16, 5, 2, 47, 8, 5, 47, 10, 8, 6, 47, 18, 4, 5, 2, 47, 17, 3, 8, 2, 47, 15, 47, 7, 4, 2, 47, 11, 49, 45, 4, 21, 5, 2, 47, 23, 15, 4, 7, 47, 10, 6, 2, 4, 2, 47, 13, 5, 8, 2, 47, 23, 15, 4, 9, 2, 47, 3, 47, 8, 4, 2, 3

48